# Recommendation System Via Hybrid Comparison  
#### <span style="color: cornflowerblue">Team 01 | CSPB 4502 | 11/16/22</span>

The following is a two-stage filtering of raw data for the Netflix recommendation competition to retain only users sufficiently similar to the randomly chosen subscriber (represented by *user_id*). The first stage filters by overlap of movies rated. The second filters by correlation of ratings.

The threshold for the first filter is a percentage of movies rated by *user_id* that another user has rated.

The threshold for the second filter is a correlation coefficient between the ratings of *user_id* and a user for only movies they both have rated.

Threshold selections can be found at the top of the program.

In [1]:
import pandas as pd

rating_filepath = r'../data/ratings.csv'

movie_thresh = 0.5    # threshold proportion of random_ID's movies that a user must have rated to be retained
rho_thresh = 0.4   # threshold a user's correlation coefficient must meet to be retained

In [3]:
# runtime: 20 sec
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df = pd.read_csv(rating_filepath, index_col=0)

In [4]:
df.head(10)

,Movie_Id,CustomerID,Rating,Date
79334307,14358,6,2,2005-12-04
32652672,5926,6,4,2005-10-26
38173947,6797,6,3,2004-11-10
20618167,3905,6,3,2005-12-04
99601160,17560,6,3,2004-11-10
75553656,13651,6,3,2004-11-13
29531350,5401,6,2,2004-12-14
40752419,7230,6,5,2005-08-29
69300001,12560,6,3,2005-12-04
89710890,15941,6,3,2005-03-20


In [5]:
len(df)
# Sanity check: total number of ratings before any thresholds applied

100480507

In [6]:
max_value = len(df["CustomerID"].unique())
max_value
# Output: total number of users in data frame (highest ID number)

480189

In [7]:
# runtime: 3 sec
# choose randomly a user for whom to give recommendations
random_id = int(df["CustomerID"].sample(1))
random_id

##################################################
####### Randomly selected user's ID number #######
##################################################

847615

In [8]:
# Pulling off the records of just the target user
user_df = df[df["CustomerID"] == random_id]
user_df.head(10)
len(user_df)

#################################################
##### Number of movies rated by random_user #####
#################################################

791

In [9]:
# Putting the movies watched by random_user into a list
movies_watched = user_df["Movie_Id"].tolist()

# Check: format, sanity check
print(movies_watched[0:10])


[2757, 12704, 11607, 12244, 13636, 2290, 5528, 16869, 4369, 17053]


In [10]:
# Runtime: 2 seconds
movies_watched_df = df[df["Movie_Id"].isin(movies_watched)]


# Output: all rows referring to a movie random_id rated
movies_watched_df.head(20)

,Movie_Id,CustomerID,Rating,Date
79334307,14358,6,2,2005-12-04
32652672,5926,6,4,2005-10-26
38173947,6797,6,3,2004-11-10
99601160,17560,6,3,2004-11-10
29531350,5401,6,2,2004-12-14
21282866,3962,6,4,2005-08-29
85209680,15124,6,5,2004-03-09
74786008,13534,6,4,2004-09-27
23920753,4472,6,3,2004-12-07
92320246,16377,6,4,2004-10-14


In [11]:
user_movie_count = movies_watched_df.groupby(["CustomerID"]).Movie_Id.count()
user_movie_count.head(10)
# Output: number of moviews a user has rated that randome_user has also rated
# Check: Most users will have rated at least one of random_user's rated movies

CustomerID
6     198
7     306
8      37
10    110
25      8
33     19
42     51
59     62
79    243
83     12
Name: Movie_Id, dtype: int64

In [12]:
############################################
####### Overlap threshold applied here #####
############################################

# We will pull off only users who have rated at least this proportion of the movies the target user has rated.
m_count = movie_thresh*len(movies_watched)
m_count

# Output: movie count threshold for retention of a user

395.5

In [13]:
user_movie_count = user_movie_count.reset_index()
user_movie_count.columns = ["CustomerID", "movie_count"]

# Choosing 50% instead of 60% makes a huge difference in number of records that finally get through.

# choose a ratio of 0.50. User ids who watched at least 50 percent of target user's movies 
users_same_movies = user_movie_count[user_movie_count["movie_count"] > m_count].sort_values("movie_count", ascending=False)
users_same_movies.nunique()

# Output: total number of users that rated at least 50% of movies rated by random_user

CustomerID     3928
movie_count     244
dtype: int64

In [14]:
# Sanity check: movie_count should be no higher than len(movies_watched) above
users_same_movies.head(10)

,CustomerID,movie_count
152372,847615,791
69656,387418,790
54736,305344,784
437012,2439493,783
379510,2118461,775
298354,1664010,753
128820,716173,736
235896,1314869,729
368383,2056022,723
272567,1519378,699


In [16]:
similar_users = users_same_movies["CustomerID"].tolist()
similar_users_df = df[df["CustomerID"].isin(similar_users)]

# Sanity check: ALL ratings of users heavily overlapping random_id's movies
similar_users_df.head(20)

,Movie_Id,CustomerID,Rating,Date
38942962,6923,1333,2,2004-03-07
50690952,9160,1333,2,2004-02-08
20913277,3936,1333,1,2005-07-09
95259246,16901,1333,1,2004-02-18
34789951,6236,1333,3,2004-04-06
33480490,6042,1333,2,2004-09-13
48071975,8651,1333,3,2004-05-18
57505866,10501,1333,3,2005-04-05
2500504,468,1333,2,2004-04-28
99672151,17575,1333,2,2004-02-16


In [17]:
# Number of ratings by users that have strong movie overlap with random_id
# Compare to len(df) above (about 1/4?)
len(similar_users_df)

7097618

In [18]:
# Create CustomerID vs Movie_Id pivot talbe so that
# corr() can find pairwise correlation between columns.

# Checking the starting format 
print(user_df.head(10))
del(user_df)

In [19]:
# Sanity check: movie_watched_df should include all of random_id's records
movies_watched_df[movies_watched_df["CustomerID"]==random_id]

,Movie_Id,CustomerID,Rating,Date
14268193,2757,847615,3,2005-12-11
70119852,12704,847615,4,2005-05-04
63566708,11607,847615,5,2005-08-21
67153505,12244,847615,3,2005-04-12
75364944,13636,847615,3,2005-03-08
11847479,2290,847615,4,2005-04-12
30233367,5528,847615,3,2005-05-06
94879723,16869,847615,4,2005-12-11
23331602,4369,847615,4,2005-04-12
96147575,17053,847615,4,2005-04-12


In [20]:
# Create a Movie_Id vs. CustomerID pivot table
# Running time: immediate
movies_watched_df2 = movies_watched_df[movies_watched_df['CustomerID'].isin(similar_users)]
movies_watched_df3 = movies_watched_df2.drop(['Date'], axis=1)
movies_watched_df_pivot = movies_watched_df3.pivot(index='CustomerID', columns='Movie_Id')

In [21]:
# Check: No more than about 40% of ratings are NaN
movies_watched_df_pivot.head(10)

Rating                                                              \
Movie_Id    8     28    30    175   187   191   241   256   269   290   299     
CustomerID                                                                      
1333          3.0   2.0   3.0   3.0   3.0   1.0   3.0   3.0   NaN   4.0   1.0   
2213          NaN   5.0   NaN   5.0   NaN   5.0   3.0   1.0   NaN   NaN   3.0   
3321          1.0   4.0   5.0   NaN   NaN   4.0   NaN   NaN   NaN   4.0   4.0   
3998          NaN   5.0   5.0   NaN   5.0   5.0   5.0   NaN   4.0   5.0   5.0   
4597          NaN   3.0   2.0   5.0   2.0   NaN   4.0   5.0   2.0   4.0   3.0   
4905          NaN   2.0   3.0   4.0   1.0   5.0   NaN   NaN   5.0   3.0   3.0   
5530          NaN   NaN   4.0   5.0   NaN   4.0   4.0   NaN   NaN   4.0   NaN   
5652          3.0   4.0   4.0   NaN   NaN   NaN   5.0   3.0   3.0   4.0   5.0   
5980          3.0   NaN   5.0   3.0   4.0   NaN   NaN   3.0   NaN   5.0   NaN   
6206          NaN   NaN   4.0   NaN   4.0   4.0   NaN   NaN   4.0   NaN   3.0   

                                                                              \
Movie_Id   312   313   329   331   348   353   357   361   413   416   424     
CustomerID                                                                     
1333         3.0   3.0   3.0   2.0   NaN   NaN   5.0   3.0   3.0   NaN   3.0   
2213         4.0   5.0   4.0   3.0   NaN   NaN   2.0   4.0   NaN   NaN   NaN   
3321         NaN   NaN   4.0   NaN   3.0   2.0   2.0   3.0   1.0   NaN   NaN   
3998         5.0   5.0   5.0   5.0   5.0   NaN   5.0   5.0   NaN   NaN   NaN   
4597         3.0   NaN   4.0   4.0   5.0   NaN   4.0   NaN   3.0   4.0   5.0   
4905         4.0   2.0   4.0   4.0   NaN   2.0   NaN   NaN   NaN   NaN   NaN   
5530         5.0   NaN   4.0   5.0   NaN   NaN   NaN   NaN   3.0   4.0   5.0   
5652         3.0   NaN   1.0   NaN   3.0   NaN   4.0   NaN   1.0   NaN   NaN   
5980         3.0   3.0   NaN   4.0   NaN   3.0   4.0   5.0   5.0   3.0   NaN   
6206         3.0   4.0   4.0   4.0   NaN   NaN   4.0   NaN   NaN   NaN   NaN   

                                                                              \
Movie_Id   457   473   482   494   502   554   571   599   689   758   788     
CustomerID                                                                     
1333         2.0   4.0   1.0   NaN   NaN   NaN   4.0   3.0   3.0   2.0   3.0   
2213         NaN   3.0   3.0   NaN   NaN   3.0   NaN   NaN   NaN   3.0   4.0   
3321         4.0   NaN   NaN   4.0   NaN   NaN   4.0   4.0   NaN   4.0   NaN   
3998         5.0   5.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   5.0   4.0   
4597         3.0   2.0   NaN   4.0   NaN   3.0   5.0   NaN   NaN   3.0   5.0   
4905         5.0   NaN   4.0   NaN   NaN   NaN   1.0   3.0   NaN   4.0   2.0   
5530         5.0   4.0   NaN   5.0   NaN   NaN   5.0   4.0   NaN   NaN   5.0   
5652         3.0   4.0   5.0   5.0   NaN   NaN   4.0   NaN   NaN   4.0   2.0   
5980         5.0   4.0   NaN   5.0   NaN   3.0   5.0   NaN   NaN   5.0   5.0   
6206         5.0   NaN   4.0   5.0   NaN   NaN   4.0   NaN   NaN   4.0   5.0   

                                                                              \
Movie_Id   831   886   896   989   994   1068  1144  1145  1180  1202  1255    
CustomerID                                                                     
1333         NaN   4.0   2.0   3.0   NaN   4.0   2.0   NaN   4.0   NaN   2.0   
2213         4.0   NaN   5.0   NaN   4.0   NaN   4.0   4.0   5.0   5.0   4.0   
3321         4.0   5.0   3.0   NaN   NaN   1.0   NaN   NaN   4.0   4.0   4.0   
3998         NaN   NaN   4.0   NaN   5.0   NaN   NaN   5.0   5.0   2.0   5.0   
4597         NaN   4.0   NaN   3.0   NaN   NaN   NaN   1.0   2.0   3.0   3.0   
4905         4.0   NaN   NaN   NaN   NaN   NaN   3.0   4.0   2.0   NaN   4.0   
5530         NaN   5.0   NaN   NaN   4.0   4.0   3.0   NaN   3.0   4.0   4.0   
5652         NaN   3.0   NaN   NaN   NaN   NaN   4.0   NaN   4.0   2.0   Na

In [22]:
# Pivot, to prepare to correlate columns
corr_df = movies_watched_df_pivot.transpose()


In [23]:
# Data cleaning: corrwith ignores NaN
corr_df2 = corr_df.corrwith(corr_df[random_id], method='pearson')
# running time: 2 secs

In [24]:
# Unsorted
corr_df2.head(10)

CustomerID
1333    0.080487
2213   -0.118810
3321    0.014156
3998    0.053834
4597    0.197528
4905    0.090901
5530    0.335341
5652    0.207252
5980    0.047062
6206    0.188541
dtype: float64

In [25]:
# Sanity check: Count of unique users with sufficient 
# overlap matches length of user_same_movies above
corr_df3 = corr_df2.sort_values(ascending=False).drop_duplicates()
corr_df3.shape

(3927,)

In [26]:
# Sanity check: only random_id has correlation 
# coefficient of 1; only random_id with small
# count of ratings would have correlation
# coefficients approaching 0.8
corr_df3.head(10)

CustomerID
847615     1.000000
2316081    0.471131
1075030    0.444794
1955660    0.437830
1308242    0.430500
752619     0.421297
774749     0.419274
653906     0.418695
1486688    0.417526
845356     0.415682
dtype: float64

In [27]:
# Uninteresting formatting
# Convert series to dataframe
corr_df4 = corr_df3.to_frame()


In [28]:
# Check: formatting
corr_df4.rename( columns={0:'corr'}, inplace=True )
corr_df4.head(10)

,corr
CustomerID,
847615,1.000000
2316081,0.471131
1075030,0.444794
1955660,0.437830
1308242,0.430500
752619,0.421297
774749,0.419274
653906,0.418695
1486688,0.417526


In [29]:
##########################################################
##### Correlation coefficient threshold applied here #####
##########################################################
# Pull off users that are highly correlated in ratings
# True would prevent CustomerID index from being retained as a column
corr_df5 = corr_df4.drop(axis=0, index = random_id)
top_users = corr_df5[(corr_df5["corr"] >= rho_thresh)]
top_users.head(10)

,corr
CustomerID,
2316081,0.471131
1075030,0.444794
1955660,0.437830
1308242,0.430500
752619,0.421297
774749,0.419274
653906,0.418695
1486688,0.417526
845356,0.415682


In [30]:
top_users.size

# Number of users left after applying movie-overlap and rating-correlation filters
# Compare to max_value above

18

In [31]:
# Uninteresting reformatting
top_users.reset_index(drop=False, inplace=True)

# Check: format to use isin(): sequential index, two labeled columns
top_users.head(10)

,CustomerID,corr
0,2316081,0.471131
1,1075030,0.444794
2,1955660,0.437830
3,1308242,0.430500
4,752619,0.421297
5,774749,0.419274
6,653906,0.418695
7,1486688,0.417526
8,845356,0.415682
9,631075,0.414541


In [32]:
# top_users['CustomerID'].astype(int)
# similar_users_df['CustomerID'].astype(int)
# top_users.dtypes

In [33]:
# Pull off all highly correleted highly overlapping users from original dataframe with full detail
similar_users_df2 = similar_users_df[similar_users_df['CustomerID'].isin(top_users['CustomerID'])]
similar_users_df2.head(10)


,Movie_Id,CustomerID,Rating,Date
90160736,16062,474700,3,2003-07-22
30097927,5510,474700,4,2004-11-15
23500481,4393,474700,2,2005-05-12
77083580,14031,474700,5,2004-12-02
20384315,3865,474700,1,2003-09-11
37656841,6702,474700,3,2003-07-31
6841637,1336,474700,2,2003-09-11
76240143,13795,474700,3,2004-09-24
37638515,6697,474700,3,2004-04-25
46399977,8334,474700,3,2003-07-23


In [34]:
# Merge in correlation coefficient dictionary, for later credibility weighting of ratings
# merge is preferred to concat for columns
similar_users_df3 = pd.merge(similar_users_df2, top_users, how='inner')
similar_users_df3.head(10)

,Movie_Id,CustomerID,Rating,Date,corr
0,16062,474700,3,2003-07-22,0.403308
1,5510,474700,4,2004-11-15,0.403308
2,4393,474700,2,2005-05-12,0.403308
3,14031,474700,5,2004-12-02,0.403308
4,3865,474700,1,2003-09-11,0.403308
5,6702,474700,3,2003-07-31,0.403308
6,1336,474700,2,2003-09-11,0.403308
7,13795,474700,3,2004-09-24,0.403308
8,6697,474700,3,2004-04-25,0.403308
9,8334,474700,3,2003-07-23,0.403308


## __Parameter selection__

The goal is to whittle down the initial 20 million records to about half a million. If the output is far fewer, the filter parameters for correlation and movie count can be lowered and raised respectively and the program restarted *after* the point at which *user_id* is generated.

Note that if *user_id* has rated few movies (less than 100), correlation coefficients can be extreme, either very high (approaching 0.8) or very low (never higher than 0.25).

While high $\rho$ may be misleading in cases of small movie counts, the *relative* values between users are what would used for credibility weighting, i.e., the use of including correclation coefficients in the output file is to differentiate credibility of different users' ratings in predicting *user_id*'s taste.

In [35]:
# Sanity check: sufficient number of ratings left (millions? 20,000 is too low)
similar_users_df3.shape

(24583, 5)

In [37]:
similar_users_df3.to_csv("../data/rating_corr.csv",header=True, index=False)


#### References:

https://medium.com/codex/hybrid-recommender-system-netflix-prize-dataset-e9f6b4a875aa

https://www.kaggle.com/code/ayseymn/hybrid-recommender-system-netflix/notebook
